In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.



     -------------------------------------- 61.0/61.0 kB 540.4 kB/s eta 0:00:00
   ---------------------------------------- 15.8/15.8 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


In [2]:
# Метод реализации активационной функции relu
def relu(x):
    return np.maximum(0, x)

# Метод реализации активационной функции softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Метод расчета кросс-энтропии
def cross_entropy_loss(predictions, targets):
    epsilon = 1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    N = predictions.shape[0]
    cross_entropy = -np.sum(targets * np.log(predictions + 1e-9)) / N
    return cross_entropy

# Метод реализации One-Hot Encoding
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_targets = np.zeros((num_samples, num_classes))
    one_hot_targets[np.arange(num_samples), labels] = 1
    return one_hot_targets

# Метод реализации обучения нейронной сети
def train_neural_network(X, y, hidden_size, output_size, learning_rate, num_epochs, batch_size):
    input_size = X.shape[1]
    num_samples = X.shape[0]

    # Инициализация весов
    weights_input_hidden = np.random.randn(input_size, hidden_size)
    biases_hidden = np.zeros((1, hidden_size))
    weights_hidden_output = np.random.randn(hidden_size, output_size)
    biases_output = np.zeros((1, output_size))

    for epoch in range(num_epochs):
        # Перемешивание данных
        indices = np.arange(num_samples)
        np.random.shuffle(indices)

        total_loss = 0
        correct_predictions = 0

        for batch_start in range(0, num_samples, batch_size):
            # Получение пачки данных
            batch_indices = indices[batch_start:batch_start + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]

            # Прямой проход
            hidden_layer_input = np.dot(X_batch, weights_input_hidden) + biases_hidden
            hidden_layer_output = relu(hidden_layer_input)
            output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
            output_layer_output = softmax(output_layer_input)

            # Расчет ошибки
            loss = cross_entropy_loss(output_layer_output, y_batch)
            total_loss += loss

            # Расчет accuracy
            predictions = np.argmax(output_layer_output, axis=1)
            true_labels = np.argmax(y_batch, axis=1)
            correct_predictions += np.sum(predictions == true_labels)

            # Обратный проход (обновление весов)
            output_error = output_layer_output - y_batch
            hidden_error = np.dot(output_error, weights_hidden_output.T) * (hidden_layer_output > 0)

            weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_error)
            biases_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
            weights_input_hidden -= learning_rate * np.dot(X_batch.T, hidden_error)
            biases_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

        average_loss = total_loss / (num_samples / batch_size)
        accuracy = correct_predictions / num_samples

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")

    return weights_input_hidden, biases_hidden, weights_hidden_output, biases_output

# Загрузка данных MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Преобразование изображений в одномерные векторы и нормализация значений
X_train = X_train.reshape((X_train.shape[0], -1)) / 255.0
X_test = X_test.reshape((X_test.shape[0], -1)) / 255.0

# Преобразование меток классов в One-Hot Encoding формат
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Параметры нейронной сети
hidden_size = 128
output_size = 10
learning_rate = 0.01
num_epochs = 10
min_batch_size = 64
max_batch_size = 64


# Обучение сети с разными размерами пачек данных
for batch_size in range(min_batch_size, max_batch_size + 1, 8):
    print(f"\nTraining with batch size {batch_size}\n{'=' * 40}")
    train_neural_network(X_train, y_train_one_hot, hidden_size, output_size, learning_rate, num_epochs, batch_size)


Training with batch size 64
Epoch 1/10, Loss: 0.9525, Accuracy: 0.7896
Epoch 2/10, Loss: 0.4008, Accuracy: 0.8915
Epoch 3/10, Loss: 0.3118, Accuracy: 0.9145
Epoch 4/10, Loss: 0.2637, Accuracy: 0.9269
Epoch 5/10, Loss: 0.2351, Accuracy: 0.9338
Epoch 6/10, Loss: 0.2118, Accuracy: 0.9398
Epoch 7/10, Loss: 0.1963, Accuracy: 0.9451
Epoch 8/10, Loss: 0.1800, Accuracy: 0.9494
Epoch 9/10, Loss: 0.1700, Accuracy: 0.9517
Epoch 10/10, Loss: 0.1592, Accuracy: 0.9542


In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Метод реализации активационной функции relu
def relu(x):
    return np.maximum(0, x)

# Метод реализации активационной функции softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Метод расчета кросс-энтропии
def cross_entropy_loss(predictions, targets):
    epsilon = 1e-15
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    N = predictions.shape[0]
    cross_entropy = -np.sum(targets * np.log(predictions + 1e-9)) / N
    return cross_entropy

# Метод реализации One-Hot Encoding
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_targets = np.zeros((num_samples, num_classes))
    one_hot_targets[np.arange(num_samples), labels] = 1
    return one_hot_targets

# Метод реализации обучения нейронной сети
def train_neural_network(X, y, hidden_size, output_size, learning_rate, num_epochs, batch_size):
    input_size = X.shape[1]
    num_samples = X.shape[0]

    # Инициализация весов
    weights_input_hidden = np.random.randn(input_size, hidden_size)
    biases_hidden = np.zeros((1, hidden_size))
    weights_hidden_output = np.random.randn(hidden_size, output_size)
    biases_output = np.zeros((1, output_size))

    for epoch in range(num_epochs):
        # Перемешивание данных
        indices = np.arange(num_samples)
        np.random.shuffle(indices)

        total_loss = 0
        correct_predictions = 0

        for batch_start in range(0, num_samples, batch_size):
            # Получение пачки данных
            batch_indices = indices[batch_start:batch_start + batch_size]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]

            # Прямой проход
            hidden_layer_input = np.dot(X_batch, weights_input_hidden) + biases_hidden
            hidden_layer_output = relu(hidden_layer_input)
            output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + biases_output
            output_layer_output = softmax(output_layer_input)

            # Расчет ошибки
            loss = cross_entropy_loss(output_layer_output, y_batch)
            total_loss += loss

            # Расчет accuracy
            predictions = np.argmax(output_layer_output, axis=1)
            true_labels = np.argmax(y_batch, axis=1)
            correct_predictions += np.sum(predictions == true_labels)

            # Обратный проход (обновление весов)
            output_error = output_layer_output - y_batch
            hidden_error = np.dot(output_error, weights_hidden_output.T) * (hidden_layer_output > 0)

            weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_error)
            biases_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
            weights_input_hidden -= learning_rate * np.dot(X_batch.T, hidden_error)
            biases_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

        average_loss = total_loss / (num_samples / batch_size)
        accuracy = correct_predictions / num_samples

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")

    return weights_input_hidden, biases_hidden, weights_hidden_output, biases_output

# Загрузка данных MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Преобразование изображений в одномерные векторы и нормализация значений
X_train = X_train.reshape((X_train.shape[0], -1)) / 255.0
X_test = X_test.reshape((X_test.shape[0], -1)) / 255.0

# Преобразование меток классов в One-Hot Encoding формат
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Параметры нейронной сети
hidden_size = 300
output_size = 10
learning_rate = 0.1
num_epochs = 20
min_batch_size = 8
max_batch_size = 64


# Обучение сети с разными размерами пачек данных
for batch_size in range(min_batch_size, max_batch_size + 1, 8):
    print(f"\nTraining with batch size {batch_size}\n{'=' * 40}")
    train_neural_network(X_train, y_train_one_hot, hidden_size, output_size, learning_rate, num_epochs, batch_size)


Training with batch size 8
Epoch 1/20, Loss: 1.6119, Accuracy: 0.4661
Epoch 2/20, Loss: 1.3162, Accuracy: 0.5621
Epoch 3/20, Loss: 1.2197, Accuracy: 0.5809
Epoch 4/20, Loss: 1.3396, Accuracy: 0.5141
Epoch 5/20, Loss: 1.2252, Accuracy: 0.5753
Epoch 6/20, Loss: 1.1956, Accuracy: 0.5837
Epoch 7/20, Loss: 1.2230, Accuracy: 0.5706
Epoch 8/20, Loss: 1.2312, Accuracy: 0.5571
Epoch 9/20, Loss: 1.2683, Accuracy: 0.5437
Epoch 10/20, Loss: 1.2319, Accuracy: 0.5586
Epoch 11/20, Loss: 1.2986, Accuracy: 0.5218
Epoch 12/20, Loss: 1.2609, Accuracy: 0.5353
Epoch 13/20, Loss: 1.2520, Accuracy: 0.5321
Epoch 14/20, Loss: 1.2299, Accuracy: 0.5475
Epoch 15/20, Loss: 1.2864, Accuracy: 0.5186
Epoch 16/20, Loss: 1.2169, Accuracy: 0.5514
Epoch 17/20, Loss: 1.3043, Accuracy: 0.5140
Epoch 18/20, Loss: 1.2482, Accuracy: 0.5359
Epoch 19/20, Loss: 1.3363, Accuracy: 0.4946
Epoch 20/20, Loss: 1.2949, Accuracy: 0.5172

Training with batch size 16
Epoch 1/20, Loss: 1.9941, Accuracy: 0.2769
Epoch 2/20, Loss: 1.8528, Acc